In [26]:
# Importação de Bibliotecas

import requests#Permiti fazer uma requisição HTTPS na API do OpenWeather

import pandas as pd #Usado para analise e manipulção de dados, especialmente com DataFrame

from sklearn.linear_model import LinearRegression #Usado para treinar o modelo de regressão linear 


In [27]:
# Função para Obter Dados da API

def obter_dados_cidade(cidade): #função que vai permitir a busca de uma cidade especifica
    api_key = '1dc1f784d3abea26b268cf151165db0f'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={api_key}&units=metric'
    response = requests.get(url)

    # Verificando se a resposta da API foi bem executada
    if response.status_code != 200: # Esse codigo indica suceeso.
        print(f'Erro ao obter dados da API: {response.status_code} -  {response.text}')
        return None
    
    return response.json() # Se a resposta houver sucesso, retorna os dados em formato JSON para facilitar o entendimento 

In [28]:
# Função para preparar os dados

def preparar_dados(dados): # Extrai informações úteis retornado pela API
    if dados is None:
        return None # Adicionada para garantir que os dados não sejam inválidos ou nulos
    
    cidade = dados['name']

    # O campo main contem os dados climáticos e ao lado são os dados extraidos do campo main 
    temperatura = dados['main']['temp'] 
    umidade = dados['main']['humidity']
    pressao = dados['main']['pressure']
    descricao_clima = dados['weather'][0]['description'] 

    # Criação de um DataFrame, pois preciso dos meus dados estruturados similar a uma tabela 
    df = pd.DataFrame({ # Função que criou meu DataFrame
        'cidade' : [cidade],
        'temperatura' : [temperatura],
        'umidade' : [umidade],
        'pressao' : [pressao],
        'descricao_clima' : [descricao_clima]   
    })

    return df

In [29]:
 # Função para treinar o modelo
def treinar_modelo(df):
    if 'temperatura_futura' not in df.columns or df['temperatura_futura'].isnull().any():
        raise ValueError("A coluna 'temperatura_futura' deve estar presente e não pode conter valores NaN.") # Essa coluna é fundamental, pois contém os valores que o modelo vai tentar prever

    X = df[['temperatura', 'umidade', 'pressao']]
    y = df['temperatura_futura']
    model = LinearRegression()
    model.fit(X, y)  # Adicionado para aprender a relação entre x (as variáveis preditivas) e y (o que queremos prever)

    
    return model


In [30]:
# Função para Fazer a Previsão baseado no modelo treinado

def prever_temperatura(model,dados_reais):
    nova_previsao = model.predict([dados_reais]) # Adicionado para verificar se os dados estão no formato corretos 
    return nova_previsao[0]

In [31]:
# Célula 6: Função principal
def main():
    cidade = input("Digite o nome da cidade: ")  # Agora o usuário pode escolher a cidade
    dados = obter_dados_cidade(cidade)

    # Preparar os dados
    df_atual = preparar_dados(dados)

    if df_atual is None:
        print("Não foi possível obter os dados para a cidade especificada.")
        return

    # Dados históricos
    historico_dados = pd.DataFrame({
        'temperatura': [30, 29, 28, 27],
        'umidade': [80, 75, 70, 65],
        'pressao': [1013, 1012, 1011, 1010],
        'temperatura_futura': [31, 30, 29, 28]  # Temperaturas futuras correspondentes
    })

    # Concatenar os dados históricos com os dados obtidos
    df = pd.concat([historico_dados, df_atual], ignore_index=True) # Uma lista contendo dois df que queremos combinar e o argumento ignore_index=True permite que os não cause índices duplicados.

    # Processo útil para um modelo de regressão linear, pois  prevê o próximo valor de uma sequência (neste caso, a temperatura futura).
    df['temperatura_futura'] = df['temperatura'].shift(-1) # Cada linha contem a temperatura atual e a temperatura da próxima linha.
    df['temperatura_futura'].fillna(df['temperatura'].iloc[-1], inplace=True)

    print("Dados utilizados para treinar o modelo:")
    print(df)

    # Treinar o modelo
    try:
        modelo = treinar_modelo(df)
    except ValueError as e:
        print(f"Erro ao treinar o modelo: {e}")
        return

    # Prever a temperatura com os dados atuais
    dados_reais = [df_atual['temperatura'].iloc[0], df_atual['umidade'].iloc[0], df_atual['pressao'].iloc[0]]
    temperatura_prevista = prever_temperatura(modelo, dados_reais)

    # Exibir o resultado
    print(f"Temperatura atual em {df_atual['cidade'].iloc[0]}: {df_atual['temperatura'].iloc[0]}°C")
    print(f"Previsão de temperatura ajustada: {temperatura_prevista:.2f}°C")

# Executar o programa


In [32]:
# Célula 7: Executar o programa
main()


Dados utilizados para treinar o modelo:
   temperatura  umidade  pressao  temperatura_futura  cidade descricao_clima
0        30.00       80     1013               29.00     NaN             NaN
1        29.00       75     1012               28.00     NaN             NaN
2        28.00       70     1011               27.00     NaN             NaN
3        27.00       65     1010               15.85     NaN             NaN
4        15.85       76     1028               15.85  London   broken clouds
Temperatura atual em London: 15.85°C
Previsão de temperatura ajustada: 15.75°C


C:\Users\JúliaMelo-HXBrasil\AppData\Local\Temp\ipykernel_18344\2632886659.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['temperatura_futura'].fillna(df['temperatura'].iloc[-1], inplace=True)
c:\Users\Julia\Documents\desafio3\.env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
